In [3]:
import os
import polars as pl
import altair as alt

pl.Config.set_tbl_rows(300)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [8]:
stvo = pl.read_parquet(os.path.join("data_raw","stvo.parquet"))

In [10]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    (pl.col('datum').dt.year() >= 1918)
    & 
    (~pl.col('komora_komplet').str.contains('(?i)senát')) 
    & 
    (~pl.col('komora_komplet').str.contains('(?i)ústavo')) 
    & 
    (~pl.col('komora_komplet').str.contains('(?i)společn'))
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).with_columns(
    pl.when(pl.col("soubor") == "1996ps_stenprot_006schuz_s006024.htm").then(pl.lit(2006)).otherwise(pl.col('obdobi')).alias('obdobi')
).collect()

In [11]:
def cetnost_po_roce(popisek, hledani):
    return df.filter(
        pl.col('text').str.contains(hledani)
    ).with_columns(
        pl.col('text').str.count_matches(hledani).alias('pocet_vyskytu')
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col('pocet_vyskytu').sum()
    ).join(
        slov_po_roce, on='datum', how='right'
    ).with_columns(
        (pl.col('pocet_vyskytu') / pl.col('pocet_slov')).alias('podil')
    ).select(
        pl.col(['datum','podil'])
    ).with_columns(
        pl.lit(popisek).alias('co')
    )

In [12]:
slov_po_roce = df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('pocet_slov').sum())

In [30]:
", ".join(stvo.filter(pl.col('text_merged').str.contains('[řě]')).to_series().to_list())

'představenstvo, kněžstvo, zvěrstvo, zemědělstvo, příbuzenstvo, měšťanstvo, malozemědělstvo, těžařstvo, spotřebitelstvo, čtenářstvo, člověčenstvo, lékařstvo, těžířstvo, pořadatelstvo, legionářstvo, ředitelstvo, zaměstnavatelstvo, křesťanstvo, domkářstvo, vinařstvo, vysokoprevoschoditělstvo, představitelstvo'

In [ ]:
", ".join([stvo.to_series().to_list())

In [26]:
alt.Chart(
    cetnost_po_roce("člověčenstvo","(?i)člověčenstvo").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [34]:
alt.Chart(
    cetnost_po_roce("čtenářstvo","(?i)čtenářstvo").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [36]:
alt.Chart(
    cetnost_po_roce("kněžstvo","(?i)kněžstvo").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [38]:
alt.Chart(
    cetnost_po_roce("domkářstvo","(?i)domkářstvo").to_pandas(),
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("podil:Q")
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)